<a href="https://colab.research.google.com/github/gbrlpzz/liminal-ai-prompts/blob/main/encouraging_reuse_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [41]:
import os
import json
from PIL import Image
import google.generativeai as genai
from google.colab import userdata
import re
print(f"google-generativeai version: {genai.__version__}")

google-generativeai version: 0.7.2


Load Drive

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


API Key

In [43]:
GOOGLE_API_KEY = userdata.get('KEY')
genai.configure(api_key=GOOGLE_API_KEY)
print(f"API Key (first 5 chars): {GOOGLE_API_KEY[:5]}...")

API Key (first 5 chars): AIzaS...


Test Connection and Load Model List

In [44]:
try:
    models = list(genai.list_models())
    print(f"Number of models available: {len(models)}")
    for model in models:
        print(f"- {model.name}")
except Exception as e:
    print(f"Error listing models: {str(e)}")

Number of models available: 20
- models/chat-bison-001
- models/text-bison-001
- models/embedding-gecko-001
- models/gemini-1.0-pro-latest
- models/gemini-1.0-pro
- models/gemini-pro
- models/gemini-1.0-pro-001
- models/gemini-1.0-pro-vision-latest
- models/gemini-pro-vision
- models/gemini-1.5-pro-latest
- models/gemini-1.5-pro-001
- models/gemini-1.5-pro
- models/gemini-1.5-pro-exp-0801
- models/gemini-1.5-flash-latest
- models/gemini-1.5-flash-001
- models/gemini-1.5-flash
- models/gemini-1.5-flash-001-tuning
- models/embedding-001
- models/text-embedding-004
- models/aqa


Load Model

In [45]:
try:
    model = genai.GenerativeModel('gemini-1.5-pro-latest')
    print("Model initialized successfully")
except Exception as e:
    print(f"Error initializing model: {str(e)}")

Model initialized successfully


Analyze Image

In [46]:
def analyze_image(image):
    try:
        model = genai.GenerativeModel('gemini-1.5-pro-latest')
        prompt = """
        Analyze this image of a building and answer the following questions. For each question, choose ONLY from the provided options without providing any additional text:

        1. Person present: FALSE or TRUE
        2. Dogs/Pets present: FALSE or TRUE
        3. Livestock present: FALSE or TRUE
        4. Sounds present inside: FALSE or TRUE
        5. Lights present inside: FALSE or TRUE
        6. Collapsed Roof: FALSE or TRUE
        7. Windows: "No - Close (if all)" or "Yes - Open (if any)" or "None"
        8. Window Broken: "None" or "True (if any broken)" or "False (if all windows intact)"
        9. Shutters: "None" or "Close (if all)" or "Open (if any)"
        10. Doorway Curtains: "None" or "Unsecured" or "Secured (if any)"
        11. For Sale Sign: FALSE or TRUE
        12. Address Number Condition: "None" or "Decorative / Tiled (if any)" or "Not Updated"
        13. Chimney Smoke: FALSE or TRUE
        14. Vehicle present in garage or private driveway: FALSE or TRUE
        15. Hanging laundry: FALSE or TRUE
        16. Personal Belongings Outside: "None" or "Good Condition" or "Mixed Condition" or "Bad Condition"
        17. Mail accumulation: "None" or "Minimal" or "Extensive"
        18. Plants / Gardens: "None" or "Well maintained (any)" or "Overgrown/Dead"
        19. Perceived Balcony Damage: "Mild Damage (cracking in plaster in facade)" or "No balcony" or "Severe Damage (visible rebar)" or "No Damage (good condition)"
        20. Perceived Private Staircase Damage: "No private staircases" or "Severe Damage (missing stairs)" or "No Damage (good condition)" or "Mild Damage (rusted railings)"
        21. Perceived Shutter Damage: "No Shutters" or "No Damage (good condition)" or "Mild Damage (needs to be repainted)" or "Severe Damage (rotting)"
        22. Door Board: "None" or "Good Condition (any)" or "Bad Condition"

        Provide your answers as a numbered list, using ONLY the exact phrases from the options given. DO NOT write any additional text, introduction or explanation.
        For example: 22. Door Board: "None"
        """
        response = model.generate_content([prompt, image])
        return response.text
    except Exception as e:
        print(f"Error analyzing image: {str(e)}")
        return None

Assign the image path and start the analysis

In [53]:
def process_analysis_result(result_string):
    lines = result_string.split('\n')
    result_dict = {}
    for line in lines:
        parts = line.split(': ', 1)
        if len(parts) == 2:
            key = parts[0].strip().split('. ', 1)[-1]
            value = parts[1].strip().strip('"')
            result_dict[key] = value
    return result_dict

In [54]:
def calculate_occupancy_rating(result_dict):
    point_system = {
        "Person present": {"FALSE": 0, "TRUE": 4},
        "Dogs/Pets present": {"FALSE": 0, "TRUE": 3},
        "Livestock present": {"FALSE": 0, "TRUE": 2},
        "Sounds present inside": {"FALSE": 0, "TRUE": 2},
        "Lights present inside": {"FALSE": 0, "TRUE": 3},
        "Collapsed Roof": {"FALSE": 0, "TRUE": -6},
        "Windows": {"No - Close (if all)": 0, "Yes - Open (if any)": 2, "None": -3},
        "Window Broken": {"None": 0, "True (if any broken)": -2, "False (if all windows intact)": 1},
        "Shutters": {"None": 0, "Close (if all)": -1, "Open (if any)": 2},
        "Doorway Curtains": {"None": 0, "Unsecured": 2, "Secured (if any)": 2},
        "For Sale Sign": {"FALSE": 0, "TRUE": -2},
        "Address Number Condition": {"None": 0, "Decorative / Tiled (if any)": 1, "Not Updated": 0},
        "Chimney Smoke": {"FALSE": 0, "TRUE": 3},
        "Vehicle present in garage or private driveway": {"FALSE": 0, "TRUE": 2},
        "Hanging laundry": {"FALSE": 0, "TRUE": 4},
        "Personal Belongings Outside": {"None": -1, "Good Condition": 3, "Mixed Condition": 2, "Bad Condition": -1},
        "Mail accumulation": {"None": 0, "Minimal": -1, "Extensive": -1},
        "Plants / Gardens": {"None": 0, "Well maintained (any)": 1, "Overgrown/Dead": -2},
        "Perceived Balcony Damage": {"Mild Damage (cracking in plaster in facade)": 0, "No balcony": 0, "Severe Damage (visible rebar)": -3, "No Damage (good condition)": 2},
        "Perceived Private Staircase Damage": {"No private staircases": 0, "Severe Damage (missing stairs)": -2, "No Damage (good condition)": 1, "Mild Damage (rusted railings)": 0},
        "Perceived Shutter Damage": {"No Shutters": 0, "No Damage (good condition)": 1, "Mild Damage (needs to be repainted)": 0, "Severe Damage (rotting)": -2},
        "Door Board": {"None": 0, "Good Condition (any)": 1, "Bad Condition": -2}
    }
    total_points = sum(point_system[key].get(value, 0) for key, value in result_dict.items() if key in point_system)
    return total_points

In [55]:
def process_folder(folder_path):
    results = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)
            analysis_result = analyze_image(image)
            if analysis_result:
                processed_result = process_analysis_result(analysis_result)
                results.append(processed_result)
    return results

In [56]:
def synthesize_results(results, folder_name):
    if not results:
        return {}

    synthesized = {"Building ID": folder_name}

    for key in results[0].keys():
        if key == "Building ID":
            continue

        values = [result.get(key) for result in results if result.get(key)]

        if key in ["Person present", "Dogs/Pets present", "Livestock present", "Sounds present inside",
                   "Lights present inside", "Collapsed Roof", "For Sale Sign", "Chimney Smoke",
                   "Vehicle present in garage or private driveway", "Hanging laundry"]:
            synthesized[key] = "TRUE" if "TRUE" in values else "FALSE"

        elif key in ["Windows", "Shutters"]:
            if "Yes - Open (if any)" in values or "Open (if any)" in values:
                synthesized[key] = "Yes - Open (if any)" if key == "Windows" else "Open (if any)"
            elif "No - Close (if all)" in values or "Close (if all)" in values:
                synthesized[key] = "No - Close (if all)" if key == "Windows" else "Close (if all)"
            else:
                synthesized[key] = "None"

        elif key == "Window Broken":
            if "True (if any broken)" in values:
                synthesized[key] = "True (if any broken)"
            elif "False (if all windows intact)" in values:
                synthesized[key] = "False (if all windows intact)"
            else:
                synthesized[key] = "None"

        elif key in ["Doorway Curtains", "Personal Belongings Outside", "Mail accumulation",
                     "Plants / Gardens", "Perceived Balcony Damage", "Perceived Private Staircase Damage",
                     "Perceived Shutter Damage", "Door Board"]:
            value_priority = {
                "Doorway Curtains": ["Secured (if any)", "Unsecured", "None"],
                "Personal Belongings Outside": ["Good Condition", "Mixed Condition", "Bad Condition", "None"],
                "Mail accumulation": ["Extensive", "Minimal", "None"],
                "Plants / Gardens": ["Well maintained (any)", "Overgrown/Dead", "None"],
                "Perceived Balcony Damage": ["Severe Damage (visible rebar)", "Mild Damage (cracking in plaster in facade)", "No Damage (good condition)", "No balcony"],
                "Perceived Private Staircase Damage": ["Severe Damage (missing stairs)", "Mild Damage (rusted railings)", "No Damage (good condition)", "No private staircases"],
                "Perceived Shutter Damage": ["Severe Damage (rotting)", "Mild Damage (needs to be repainted)", "No Damage (good condition)", "No Shutters"],
                "Door Board": ["Bad Condition", "Good Condition (any)", "None"]
            }
            for priority_value in value_priority[key]:
                if priority_value in values:
                    synthesized[key] = priority_value
                    break

        else:  # For any other keys not specifically handled
            synthesized[key] = max(set(values), key=values.count)

    return synthesized

In [58]:
def main(folder_path):
    folder_name = os.path.basename(folder_path)
    results = process_folder(folder_path)
    synthesized_result = synthesize_results(results, folder_name)
    occupancy_rating = calculate_occupancy_rating(synthesized_result)
    synthesized_result["Occupancy Rating"] = occupancy_rating
    return json.dumps(synthesized_result, indent=2)

In [60]:
folder_path = "/content/drive/MyDrive/Encouraging Reuse 2024: Fontainemore/100"
output = main(folder_path)
print(output)

{
  "Building ID": "100",
  "Person present": "FALSE",
  "Dogs/Pets present": "FALSE",
  "Livestock present": "FALSE",
  "Sounds present inside": "FALSE",
  "Lights present inside": "FALSE",
  "Collapsed Roof": "FALSE",
  "Windows": "Yes - Open (if any)",
  "Window Broken": "None",
  "Shutters": "Close (if all)",
  "Doorway Curtains": "None",
  "For Sale Sign": "FALSE",
  "Civic Number Condition": "None",
  "Chimney Smoke": "FALSE",
  "Vehicle present in garage or private driveway": "FALSE",
  "Hanging laundry": "FALSE",
  "Personal Belongings Outside": "None",
  "Mail accumulation": "None",
  "Plants / Gardens": "Well maintained (any)",
  "Perceived Balcony Damage": "No Damage (good condition)",
  "Perceived Private Staircase Damage": "No private staircases",
  "Perceived Shutter Damage": "No Damage (good condition)",
  "Door Board": "None",
  "Occupancy Rating": 4
}
